In [ ]:
import pandas as pd
import numpy as np

# 3 parts:
# kostenschätzung
# ausgabenberechnung (bilanz berechnen => schätzung vs. tatsächliche kosten)
# kostenvergleich

In [ ]:
from building_cost_analysis.settings import (
    DIR_ANGEBOTE,
    PATH_GEWERKE,
    PATH_AUSGABEN,
    PATH_KOSTENUEBERSICHT
)
from building_cost_analysis.parser import (
    get_df_gewerke,
    get_offer_file_paths,
    get_offer_file_paths_and_sheets,
    get_df_offer,
    get_df_offers_combined,
    get_list_df_offers
)

from building_cost_analysis.calc import (
    append_rolled_up_sum_to_df,
    get_df_duplicated_positions,
    compare_offers,
    get_missing_positions
)

# get_df_offers_combined
list_paths_sheets = get_offer_file_paths_and_sheets(DIR_ANGEBOTE)
df_gewerke = get_df_gewerke(PATH_GEWERKE)

In [ ]:
list_paths_sheets

In [ ]:
list_paths_sheets[-4]

In [ ]:
def get_paths_sheets_by_name(list_path_sheets: list, gewerk: str, sheet: str):
    for path_sheet in list_path_sheets:
        path_offer = path_sheet['path_offer'].replace('.xlsx', '')
        if path_offer.endswith(gewerk) and sheet == path_sheet['sheet_name']:
            return path_sheet



In [ ]:
# pick gewerke
list_combine_path_sheets = []
baumeister = get_paths_sheets_by_name(list_paths_sheets, "Baumeister", "Mueller_Minimum")
bautischler = get_paths_sheets_by_name(list_paths_sheets, "Bautischler", "Julia_Minimum")
bodenleger = get_paths_sheets_by_name(list_paths_sheets, "Bodenleger", "Julia_Minimum")
dachdecker = get_paths_sheets_by_name(list_paths_sheets, "Dachdecker", "Wurzrainer_Minimum_2")
fenster = get_paths_sheets_by_name(list_paths_sheets, "Fenster", "Kitzfenster_Minimum")
fliesenleger = get_paths_sheets_by_name(list_paths_sheets, "Fliesenleger", "Kostenschaetzung_Minimum")
installateur = get_paths_sheets_by_name(list_paths_sheets, "Installateur", "Kracher_Minimum")
klaer = get_paths_sheets_by_name(list_paths_sheets, "Klaeranlage", "Hechenbichler_1")
kostensch = get_paths_sheets_by_name(list_paths_sheets, "Kostenschaetzung", "Kostenschaetzung_Minimum_2")
nebenkosten = get_paths_sheets_by_name(list_paths_sheets, "Nebenkosten", "Schaetzung_Minimum")
zimmerer = get_paths_sheets_by_name(list_paths_sheets, "Zimmerer", "Foidl_Minimum")

list_combine_path_sheets.append(baumeister)
list_combine_path_sheets.append(bautischler)
list_combine_path_sheets.append(bodenleger)
list_combine_path_sheets.append(dachdecker)
list_combine_path_sheets.append(fenster)
list_combine_path_sheets.append(fliesenleger)
list_combine_path_sheets.append(installateur)
list_combine_path_sheets.append(klaer)
list_combine_path_sheets.append(kostensch)
list_combine_path_sheets.append(nebenkosten)
list_combine_path_sheets.append(zimmerer)

In [ ]:

list_combine_path_sheets

In [ ]:
df_offers = get_df_offers_combined(list_combine_path_sheets)
df_combined = append_rolled_up_sum_to_df(df_offers, df_gewerke)

df_combined.loc[df_combined['level'] == 0][['summed_up']].sum() * 1.1# 5 % sicherheitsreserve

In [ ]:
display_cols=['level', 'id_0', 'id_1', 'Beschreibung', 'Brutto', 'E-Preis', 'Einheit', 'Gesamtpreis', 'Menge', 'summed_up']

df_combined.loc[df_combined['level'] == 1][display_cols]

In [ ]:
df_combined.to_csv('./data/test.csv')

In [ ]:
df_combined[display_cols].to_csv('current.csv')

In [ ]:
df_combined.columns

In [ ]:
df_combined.columns